In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''



class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        #self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(1024, num_classes)
        #self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        #out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        #print(out.size())
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [3, 3, 1, 1])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

# test()

In [2]:
class AverageMeter(object):
    r"""Computes and stores the average and current value
    """
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, *meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def print(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    r"""Computes the accuracy over the $k$ top predictions for the specified values of k
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        # _, pred = output.topk(maxk, 1, True, True)
        # pred = pred.t()
        # correct = pred.eq(target.view(1, -1).expand_as(pred))

        # faster topk (ref: https://github.com/pytorch/pytorch/issues/22812)
        _, idx = output.sort(descending=True)
        pred = idx[:,:maxk]
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        """
        print("size of pred and correct in acc fnc: ", pred.size(), correct.size(), target.view(1, -1).expand_as(pred).size())
        print(pred, target.view(1, -1).expand_as(pred))
        """
        label=torch.zeros(10)
        res = []
        cat_label={0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0}
        deer_label={0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0}
        dog_label={0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0}
        _target=target.view(1, -1).expand_as(pred).squeeze().cpu().detach().numpy() 
        _pred=pred.squeeze().cpu().detach().numpy() 
        for idx, i in enumerate(correct.squeeze()):
            if i==False:
                label[_target[idx]]+=1
                
                if _target[idx]==3:
                    cat_label[_pred[idx]]+=1
                elif _target[idx]==4:
                    deer_label[_pred[idx]]+=1
                elif _target[idx]==5:
                    dog_label[_pred[idx]]+=1
                
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res, label, cat_label, deer_label, dog_label

In [3]:
SAVEPATH = './weight/1130resnet/'
WEIGHTDECAY = 5e-4
MOMENTUM = 0.9
BATCHSIZE = 256
LR = 0.1
EPOCHS = 1000
PRINTFREQ = 10

In [7]:
import time

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader


def main():
   model = ResNet18()

   model = model.cuda()

   model.load_state_dict(torch.load("./weight/1130resnet/"+'earlystop_check_best.pth'))

   # Check number of parameters your model
   pytorch_total_params = sum(p.numel() for p in model.parameters())
   print(f"Number of parameters: {pytorch_total_params}")
   
   if int(pytorch_total_params) > 2000000:
       print('Your model has the number of parameters more than 2 millions..')
       return
   
   normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
   """
   train_transform = transforms.Compose([
       transforms.RandomCrop(32, padding=4),
       transforms.RandomHorizontalFlip(),
       transforms.RandomAffine(30),
       transforms.ColorJitter(brightness=(0.2, 2), contrast=(0.3, 2), saturation=(0.2, 2), hue=(-0.3, 0.3)), 
       transforms.ToTensor(),
       normalize, 
   ])
   
   train_dataset = torchvision.datasets.ImageFolder(
       '/home/dhk1349/Desktop/Machine Learning/train', transform=train_transform)
   train_loader = DataLoader(train_dataset,
                             batch_size=BATCHSIZE , shuffle=True,
                             num_workers=4, pin_memory=True)
   """ 


   valid_transform = transforms.Compose([
       transforms.ToTensor(),
       normalize
   ])
    
   fake_train_dataset = torchvision.datasets.ImageFolder(
       '/home/dhk1349/Desktop/Machine Learning/train', transform=valid_transform)

   val_dataset = torchvision.datasets.ImageFolder('/home/dhk1349/Desktop/Machine Learning/valid', transform=valid_transform)
   val_loader = DataLoader(val_dataset, batch_size=BATCHSIZE, shuffle=True)
   #val_loader = DataLoader(fake_train_dataset, batch_size=BATCHSIZE, shuffle=True)

   last_top1_acc = 0

   # LOSS FOR EARLY STOPPING
   best_valid_loss = float('inf')
   early_stop_check = 0

   for epoch in range(1):
       
       valid_acc, total_list, cat, deer, dog = valid(val_loader, epoch, model)
      
        
       #print(f'\t==Valid Loss: {valid_loss:.3f} | Valid acc: {valid_acc:.3f}==\n')
       
   print("==================================================")
   print(total_list)
   print("total number of wrong samples: ", total_list.sum())

   print("cat label: ", cat)
   print("deer label: ", deer)
   print("dog label: ", dog)
    
   #print(f"Last Top-1 Accuracy: {last_top1_acc}")
   print(f"Number of parameters: {pytorch_total_params}")



def valid(val_loader, epoch, model):
   # LOSS FOR EARLYSTOPPPING
   total_loss = 0
   iter_num = 0
   val_acc = 0
   total_list=torch.zeros(10)
   with torch.no_grad():
     total_cat_label={0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0}
     total_deer_label={0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0}
     total_dog_label={0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0}
     for i, (input, target) in enumerate(val_loader):
       input = input.cuda()
       target = target.cuda()

       output = model(input)
       #loss = criterion(output, target)
       acc1, result_lst, cat, deer, dog = accuracy(output, target, topk=(1,))
       
       total_list+=result_lst
        
       for idx, i in enumerate(cat.values()):
          total_cat_label[idx]+=i
       for idx, i in enumerate(deer.values()):
          total_deer_label[idx]+=i
       for idx, i in enumerate(dog.values()):
          total_dog_label[idx]+=i
       #print(total_list)
       #total_loss += loss.item()
       iter_num += 1
       val_acc += acc1[0].item()

     return  val_acc / iter_num, total_list, total_cat_label, total_deer_label, total_dog_label


if __name__ == "__main__":
    main()

Number of parameters: 1974090
tensor([1033., 1298., 1638., 2342., 2162., 2820.,  901., 1347., 1097., 1670.])
total number of wrong samples:  tensor(16308.)
cat label:  {0: 82, 1: 28, 2: 342, 3: 0, 4: 348, 5: 1007, 6: 324, 7: 88, 8: 78, 9: 45}
deer label:  {0: 77, 1: 22, 2: 385, 3: 397, 4: 0, 5: 549, 6: 147, 7: 456, 8: 102, 9: 27}
dog label:  {0: 65, 1: 50, 2: 375, 3: 1093, 4: 569, 5: 0, 6: 161, 7: 359, 8: 98, 9: 50}
Number of parameters: 1974090
